In [1]:
import csv as csv
import numpy as np

csv_file_object = csv.reader(open('train.csv', 'rb'))       # Load in the csv file
header = csv_file_object.next()                             # Skip the fist line as it is a header
data=[]                                                     # Create a variable to hold the data

In [2]:
for row in csv_file_object:                 # Skip through each row in the csv file
    data.append(row)                        # adding each row to the data variable
data = np.array(data)                       # Then convert from a list to an array

In [3]:
# So we add a ceiling
fare_ceiling = 40
# then modify the data in the Fare column to = 39, if it is greater or equal to the ceiling
data[ data[0::,9].astype(np.float) >= fare_ceiling, 9 ] = fare_ceiling - 1.0

fare_bracket_size = 10
number_of_price_brackets = fare_ceiling / fare_bracket_size

# I know there were 1st, 2nd and 3rd classes on board
number_of_classes = 3

# But it's better practice to calculate this from the data directly
# Take the length of an array of unique values in column index 2
number_of_classes = len(np.unique(data[0::,2])) 

# Initialize the survival table with all zeros
survival_table = np.zeros((2, number_of_classes, number_of_price_brackets))

In [4]:
for i in xrange(number_of_classes):       #loop through each class
  for j in xrange(number_of_price_brackets):   #loop through each price bin

    women_only_stats = data[ (data[0::,4] == "female")
                       &(data[0::,2].astype(np.float) 
                             == i+1)                       
                       &(data[0:,9].astype(np.float)  
                            >= j*fare_bracket_size)               
                       &(data[0:,9].astype(np.float)  
                            < (j+1)*fare_bracket_size) 
                          , 1]                                                 


    men_only_stats = data[ (data[0::,4] != "female")    
                       &(data[0::,2].astype(np.float) 
                             == i+1)                                       
                       &(data[0:,9].astype(np.float)   
                            >= j*fare_bracket_size)                 
                       &(data[0:,9].astype(np.float)  
                            < (j+1)*fare_bracket_size)    
                          , 1]
    survival_table[0,i,j] = np.mean(women_only_stats.astype(np.float)) 
    survival_table[1,i,j] = np.mean(men_only_stats.astype(np.float))

/home/yohoos/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/yohoos/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
survival_table[ survival_table != survival_table ] = 0
print survival_table

[[[ 0.          0.          0.83333333  0.97727273]
  [ 0.          0.91428571  0.9         1.        ]
  [ 0.59375     0.58139535  0.33333333  0.125     ]]

 [[ 0.          0.          0.4         0.38372093]
  [ 0.          0.15873016  0.16        0.21428571]
  [ 0.11153846  0.23684211  0.125       0.24      ]]]


In [6]:
survival_table[ survival_table < 0.5 ] = 0
survival_table[ survival_table >= 0.5 ] = 1 
print survival_table

[[[ 0.  0.  1.  1.]
  [ 0.  1.  1.  1.]
  [ 1.  1.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [7]:
test_file = open('test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()
predictions_file = open("genderclassfaremodel.csv", "wb")
p = csv.writer(predictions_file)
p.writerow(["PassengerId", "Survived"])

In [8]:
for row in test_file_object:                 # We are going to loop
                                              # through each passenger
                                              # in the test set                     
    for j in xrange(number_of_price_brackets):  # For each passenger we
                                              # loop thro each price bin
        try:                                      # Some passengers have no
                                                  # Fare data so try to make
          row[8] = float(row[8])                  # a float
        except:                                   # If fails: no data, so 
          bin_fare = 3 - float(row[1])            # bin the fare according Pclass
          break                                   # Break from the loop
        if row[8] > fare_ceiling:              # If there is data see if
                                                  # it is greater than fare
                                                  # ceiling we set earlier
          bin_fare = number_of_price_brackets-1   # If so set to highest bin
          break                                   # And then break loop
        if row[8] >= j * fare_bracket_size\
           and row[8] < \
           (j+1) * fare_bracket_size:             # If passed these tests 
                                                  # then loop through each bin 
          bin_fare = j                            # then assign index
          break

    if row[3] == 'female':                             #If the passenger is female
        p.writerow([row[0], "%d" % \
                   int(survival_table[0, float(row[1])-1, bin_fare])])
    else:                                          #else if male
        p.writerow([row[0], "%d" % \
                   int(survival_table[1, float(row[1])-1, bin_fare])])
     
# Close out the files.
test_file.close() 
predictions_file.close()

/home/yohoos/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/yohoos/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [9]:
print survival_table

[[[ 0.  0.  1.  1.]
  [ 0.  1.  1.  1.]
  [ 1.  1.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]
